In [1]:
from pathlib import Path
from typing import assert_never

import pandas as pd

# Pré-processamento dos dados

> Esse notebook lê os dados que foram raspados e pré-processa-os para serem utilizados em visualizações.

## Carregando os dados

O arquivo `output.json` não está presente no repositório, ele pode ser obtido através da execução do scrap, para executa-lo basta seguir as instruções do README.md.

Uma vez que o arquivo esteja presente essa seção carregará os dados no formato pandas dataframe.

In [2]:
scrap_file = Path('./nbs/output.json')

In [3]:
assert scrap_file.exists(), "Arquivo com dados raspados não foi encontrado"

In [4]:
df = pd.read_json(scrap_file)
df.head()

,codigo,disciplina,criacao,n_creditos,carga_teorica,carga_pratica,carga_estudo,duracao,carga_total,docentes_responsaveis,...,conteudo,bibliografia,idioma,oferecimento,codigo_area_concentracao,area_concentracao,codigo_commissao,commissao,codigo_programa,nome_programa
0,DCV5969,Projeto de Reforma do Código Civil,22/05/2024,8.0,2.0,2.0,4.0,15 semanas,120 horas,,...,1) Codificar ou não codificar? Eis a questão; ...,"BIANCA, Cesare Massimo. L'obbligazione nelle ...",Português,Presencial,2131,Direito Civil,2,Faculdade de Direito,2001,Direito
1,DCV5963,Direito Notarial e Registral: Novas Perspectiv...,17/11/2021,8.0,2.0,2.0,4.0,15 semanas,120 horas,,...,A disciplina será estruturada em aulas exposit...,"ALVIM NETO, José Manoel Arruda; CLÁPIS, Alexan...",Português,Presencial,2131,Direito Civil,2,Faculdade de Direito,2001,Direito
2,DCV5962,Temas Atuais de Família e Sucessão,03/05/2021,8.0,2.0,2.0,4.0,15 semanas,120 horas,Claudio Luiz Bueno de Godoy | Fernando Campos ...,...,A disciplina será formada tanto por aulas expo...,"ABRAMS, Douglas E., Contemporary Family Law, 4...",Português,Presencial,2131,Direito Civil,2,Faculdade de Direito,2001,Direito
3,DCV5961,Direito da Arte e Patrimônio Cultural,28/06/2023,8.0,2.0,2.0,4.0,15 semanas,120 horas,Silmara Juny de Abreu Chinellato | Antonio Car...,...,Ementa \nDireito da Arte. Multidisciplinarieda...,"ASCENSÃO, José de Oliveira. Direito autoral. 2...",Português,Presencial,2131,Direito Civil,2,Faculdade de Direito,2001,Direito
4,DCV5955,Dogmática e Crítica da Jurisprudência,23/04/2025,8.0,2.0,2.0,4.0,15 semanas,120 horas,Alexandre de Moraes | Otavio Luiz Rodrigues Ju...,...,§1 Introdução e apresentação da disciplina. §2...,"ÁVILA, Humberto. “Neoconstitucionalismo”: entr...",Português,Presencial,2131,Direito Civil,2,Faculdade de Direito,2001,Direito


## Filtrando os dados

Algumas disciplinas não conseguiram ser raspadas, por problemas no site do Janus. A seguir a quantidade de disciplinas:

In [5]:
amount_null = len(df[df['n_creditos'].isnull()])
amount_not_null = len(df) - amount_null
print(f'{amount_null} disciplinas não foram recuperadas corretamente. O que corresponde a {amount_null*100/len(df):.2f}% das disciplinas')

32 disciplinas não foram recuperadas corretamente. O que corresponde a 1.43% das disciplinas


O dataset final a ser avaliado não conterá as disciplinas com problemas.

In [6]:
df = df[~df['n_creditos'].isnull()]

## Validanto e transformando os tipos

Nem todos os tipos foram lidos corretamente do json. Essa seção atualiza esses campos.

In [7]:
df.dtypes

codigo                       object
disciplina                   object
criacao                      object
n_creditos                  float64
carga_teorica               float64
carga_pratica               float64
carga_estudo                float64
duracao                      object
carga_total                  object
docentes_responsaveis        object
objetivos                    object
justificativa                object
avaliacao                    object
conteudo                     object
bibliografia                 object
idioma                       object
oferecimento                 object
codigo_area_concentracao      int64
area_concentracao            object
codigo_commissao              int64
commissao                    object
codigo_programa               int64
nome_programa                object
dtype: object

In [8]:
df['criacao'] = pd.to_datetime(
    df['criacao'],
    format='%d/%m/%Y',
    errors='coerce'
)
df['carga_teorica'] = df['carga_teorica'].astype(int)
df['carga_pratica'] = df['carga_pratica'].astype(int)
df['carga_estudo'] = df['carga_estudo'].astype(int)
df['n_creditos'] = df['n_creditos'].astype(int)

As colunas `carga_total` e `duracao` são listadas em horas e semanas, respectivamente, como pode-se ver na célula a seguir. 

In [9]:
display(df['carga_total'].unique())
display(df['duracao'].unique())

array(['120 horas', '75 horas', '60 horas', '30 horas', '45 horas',
       '90 horas', '225 horas', '180 horas', '150 horas', '15 horas',
       '210 horas', '105 horas', '135 horas', '165 horas'], dtype=object)

array(['15 semanas', '4 semanas', '10 semanas', '3 semanas', '2 semanas',
       '1 semanas', '12 semanas', '6 semanas', '5 semanas', '9 semanas',
       '8 semanas', '20 semanas', '18 semanas', '7 semanas'], dtype=object)

Os seguintes comandos transformam o string em um número inteiro de horas.

In [10]:
def extrair_numero(valor_texto: str | None) -> int:
    """
    Recebe uma string como '120 horas' ou '15 semanas' e retorna apenas o valor inteiro em horas.
    """
    numero_str, tipo = str(valor_texto).split(' ')
    if tipo == 'horas':
        return int(numero_str)
    elif tipo == 'semanas':
        return 7 * 24 * int(numero_str)
    else:
        assert_never()

In [11]:
df['carga_total'] = df['carga_total'].apply(extrair_numero)
df['duracao'] = df['duracao'].apply(extrair_numero)
df['carga_total'].astype(int)
df['duracao'].astype(int)

0       2520
1       2520
2       2520
3       2520
4       2520
        ... 
2240     504
2241     336
2242     336
2243     840
2244    2016
Name: duracao, Length: 2213, dtype: int64

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2213 entries, 0 to 2244
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   codigo                    2213 non-null   object        
 1   disciplina                2213 non-null   object        
 2   criacao                   2213 non-null   datetime64[ns]
 3   n_creditos                2213 non-null   int64         
 4   carga_teorica             2213 non-null   int64         
 5   carga_pratica             2213 non-null   int64         
 6   carga_estudo              2213 non-null   int64         
 7   duracao                   2213 non-null   int64         
 8   carga_total               2213 non-null   int64         
 9   docentes_responsaveis     2213 non-null   object        
 10  objetivos                 2213 non-null   object        
 11  justificativa             2213 non-null   object        
 12  avaliacao                

## Exportando o dataframe

Para reutilizar o dataframe com os mesmos tipos e transformações específicados vamos exportar ele no formato [pickle](https://docs.python.org/3/library/pickle.html), permitindo carrega-lo mais facilmente nos notebooks seguintes.

In [13]:
df.to_pickle('nbs/output.pickle')